# 零基础入门推荐系统
[链接](https://tianchi.aliyun.com/competition/entrance/531842)。

# 任务


# 初始化 & 读取数据集

In [1]:
import os
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, from_json, greatest, lit, abs
from configparser import ConfigParser
from pyspark.sql.functions import sum as spark_sum

# 设置 JDK
os.environ["JAVA_HOME"] = "/lib/jvm/java-17-openjdk-amd64"

print(pyspark.__version__)

# 添加 jar 包
jars_dir = "/home/jovyan/jars"
jars_list = [
    os.path.join(jars_dir, f) for f in os.listdir(jars_dir) if f.endswith(".jar")
]
jars_str = ",".join(jars_list)
print(jars_str)

# 读取 MinIO 配置
config_path = os.path.expanduser("~/.minioconfig")
parser = ConfigParser()
parser.read(config_path)
endpoint = parser.get("Credentials", "endpoint")
access_key_id = parser.get("Credentials", "accessKeyID")
access_key_secret = parser.get("Credentials", "accessKeySecret")

# 创建本地 SparkSession（local 模式）
spark = (
    SparkSession.builder.appName("LocalPySparkExample")
    .config("spark.jars", jars_str)
    .master("local[*]")
    .config("spark.driver.memory", "10g")
    .config("spark.driver.maxResultSize", "4g")
    .config("spark.sql.shuffle.partitions", "100")
    .config("spark.default.parallelism", "100")
    .config("spark.executor.extraJavaOptions", "-XX:+UseG1GC -XX:InitiatingHeapOccupancyPercent=35")
    # S3A 配置
    .config("spark.hadoop.fs.s3a.endpoint", endpoint)                   # MinIO 服务地址
    .config("spark.hadoop.fs.s3a.access.key", access_key_id)            # MinIO 用户名
    .config("spark.hadoop.fs.s3a.secret.key", access_key_secret)        # MinIO 密码
    .config("spark.hadoop.fs.s3a.path.style.access", "true")            # 必须设为 true，MinIO 用路径模式
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
    .getOrCreate()
)

print('Spark Version: ', spark.version)

4.0.0
/home/jovyan/jars/hadoop-aliyun-3.4.1.jar,/home/jovyan/jars/hadoop-aws-3.4.1.jar,/home/jovyan/jars/bundle-2.32.24.jar,/home/jovyan/jars/aliyun-sdk-oss-3.18.2.jar,/home/jovyan/jars/paimon-oss-1.2.0.jar,/home/jovyan/jars/commons-configuration2-2.12.0.jar,/home/jovyan/jars/jdom2-2.0.6.1.jar
Spark Version:  4.0.0


In [16]:
!/home/linuxbrew/.linuxbrew/bin/mc ls dxp/dataset/tianchi/tianchi-news-rec/
# 同步数据到本地
!/home/linuxbrew/.linuxbrew/bin/mc cp -r dxp/dataset/tianchi/tianchi-news-rec ~/dataset/

]11;?\[2025-08-18 10:06:11 UTC] 9.9MiB STANDARD articles.csv
[2025-08-18 10:06:23 UTC] 973MiB STANDARD articles_emb.csv
[2025-08-18 10:06:12 UTC]  20MiB STANDARD testA_click_log.csv
[2025-08-18 10:06:12 UTC]  44MiB STANDARD train_click_log.csv
...ck_log.csv: 1.02 GiB / 1.02 GiB ┃▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓┃ 269.72 MiB/s 3s

In [1]:
# article_csv_path = 's3a://dataset/tianchi/tianchi-news-rec/articles.csv'
# article_df = spark.read.csv(article_csv_path, header=True)
# article_df.printSchema()

# article_emb_csv_path = 's3a://dataset/tianchi/tianchi-news-rec/articles_emb.csv'
# article_emb_df = spark.read.csv(article_emb_csv_path, header=True)
# article_emb_df.printSchema()

# train_click_csv_path = 's3a://dataset/tianchi/tianchi-news-rec/train_click_log.csv'
# train_click_df = spark.read.csv(train_click_csv_path, header=True)
# train_click_df.printSchema()

# testA_click_csv_path = 's3a://dataset/tianchi/tianchi-news-rec/testA_click_log.csv'
# testA_click_df = spark.read.csv(testA_click_csv_path, header=True)
# testA_click_df.printSchema()

# print(article_df.count(), article_emb_df.count())
# article_emb_df.show()
# article_df.show()

# 数据走读

In [1]:
import os
import sys
def get_project_path():
    import os
    parts = os.getcwd().rsplit("machine-learning", 1)
    return os.path.join(parts[0], "machine-learning")
sys.path.append(get_project_path())
sys.path.append(os.getcwd())

from internal.pyspark.pyspark_tools import SparkSessionBuilder
spark = SparkSessionBuilder().with_minio().build()
print('Spark Version: ', spark.version)

/home/jovyan/jars/hadoop-aliyun-3.4.1.jar,/home/jovyan/jars/hadoop-aws-3.4.1.jar,/home/jovyan/jars/bundle-2.32.24.jar,/home/jovyan/jars/aliyun-sdk-oss-3.18.2.jar,/home/jovyan/jars/paimon-oss-1.2.0.jar,/home/jovyan/jars/commons-configuration2-2.12.0.jar,/home/jovyan/jars/jdom2-2.0.6.1.jar
Spark Version:  4.0.0


In [2]:
from data import DataManager
dm = DataManager('~/dataset/tianchi-news-rec')

In [3]:
dm.describe()

----- article -----
          article_id    category_id  created_at_ts    words_count
count  364047.000000  364047.000000   3.640470e+05  364047.000000
mean   182023.000000     283.108239   1.474070e+12     190.897727
std    105091.461061     136.723470   4.293038e+10      59.502766
min         0.000000       0.000000   1.159356e+12       0.000000
25%     91011.500000     199.000000   1.444925e+12     159.000000
50%    182023.000000     301.000000   1.489422e+12     186.000000
75%    273034.500000     399.000000   1.509891e+12     218.000000
max    364046.000000     460.000000   1.520943e+12    6690.000000

----- article emb -----
          article_id          emb_0          emb_1          emb_2  \
count  364047.000000  364047.000000  364047.000000  364047.000000   
mean   182023.000000      -0.238645      -0.963334       0.118548   
std    105091.461061       0.451497       0.022651       0.495050   
min         0.000000      -0.991183      -0.996455      -0.968431   
25%     91011.50

In [5]:
dm.df_train_click.iloc[:100]

,user_id,click_article_id,click_timestamp,click_environment,click_deviceGroup,click_os,click_country,click_region,click_referrer_type
0,199999,160417,1507029570190,4,1,17,1,13,1
1,199999,5408,1507029571478,4,1,17,1,13,1
2,199999,50823,1507029601478,4,1,17,1,13,1
3,199998,157770,1507029532200,4,1,17,1,25,5
4,199998,96613,1507029671831,4,1,17,1,25,5
...,...,...,...,...,...,...,...,...,...
95,199973,158229,1507029591819,4,1,17,1,16,2
96,199973,160417,1507029621819,4,1,17,1,16,2
97,199972,313431,1507052104467,4,1,17,1,5,2
98,199972,199197,1507052216371,4,1,17,1,5,2


In [6]:
dm.df_train_click['user_id'].unique()

array([199999, 199998, 199997, ...,      2,      1,      0],
      shape=(200000,))

In [7]:
dm.df_test_click['user_id'].unique()

array([249999, 249998, 249997, ..., 200002, 200001, 200000],
      shape=(50000,))